In [5]:
import pandas as pd
ratings = pd.read_csv('/content/ratings.csv')
df = ratings.pivot_table(index = "userId", columns = 'movieId', values = 'rating', aggfunc = sum).values

df

<ipython-input-5-2baa1610ec85>:3: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df = ratings.pivot_table(index = "userId", columns = 'movieId', values = 'rating', aggfunc = sum).values


array([[4. , nan, 4. , ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [2.5, 2. , 2. , ..., nan, nan, nan],
       [3. , nan, nan, ..., nan, nan, nan],
       [5. , nan, nan, ..., nan, nan, nan]])

In [6]:
nan_mask = np.isnan(df) #누락값 있는 곳 미리 저장

df_filled = df.copy()

for i in range(len(df_filled)):
    mean = np.nanmean(df_filled[i])
    df_filled[i] = np.nan_to_num(df_filled[i], nan = mean)
    df_filled[i] -= mean

df_filled

array([[-0.36637931,  0.        , -0.36637931, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.63417569, -1.13417569, -1.13417569, ...,  0.        ,
         0.        ,  0.        ],
       [-0.27027027,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.31144393,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [7]:
# 코사인 유사도
def cosine_similarity_matrix(data):
    num_users = data.shape[0]
    similarity_matrix = np.zeros((num_users, num_users))

    for i in range(num_users):
        for j in range(num_users):
            if i != j:
                dot_product = np.dot(data[i], data[j])
                norm_i = np.linalg.norm(data[i])
                norm_j = np.linalg.norm(data[j])
                similarity_matrix[i][j] = dot_product / (norm_i * norm_j)

    return similarity_matrix

cosine_similarity = cosine_similarity_matrix(df_filled)

<ipython-input-7-a868343ff5b1>:12: RuntimeWarning: invalid value encountered in scalar divide
  similarity_matrix[i][j] = dot_product / (norm_i * norm_j)


In [8]:
#고려할 이웃의 수
k = 5

#각 유저별로 가장 거리가 짧았던 5명이 있는 위치
nearest_neighbors = np.argsort(cosine_similarity, axis=1)[:, 1:k+1]

In [9]:
#0번째 사람이 영화를 추천할 예정
user = 0

user_ratings = df[user]
user_nan_mask = nan_mask[user]

In [10]:
#0번째 사람이 평가하지 않은 영화 목록
unrated_indices = np.where(user_nan_mask)[0]

In [11]:
# 이웃의 인덱스 가져오기
neighbors_indices = nearest_neighbors[user]

In [13]:
#영화별 예측된 평점들을 저장해놓을 장소
predictions = {}

neibors_dindices = nearest_neighbors[user]

for movie_idx in unrated_indices:
    neighbor_ratings = [] #이웃들이 이 영화를 몇점으로 평가했는가
    for neighbor_idx in neighbors_indices:
        if not (nan_mask[neighbor_idx][movie_idx]):
            neighbor_ratings.append(df[neighbor_idx][movie_idx])

    if neighbor_ratings != []:
        predictions[movie_idx] = np.mean(neighbor_ratings)

In [18]:
movies = pd.read_csv("/content/movies.csv")
movie_names = {}
for i in range (len(movies)):
    a = movies.iloc[i]
    movie_names[a['movieId']] = a['title']

In [21]:
sorted_predicions = sorted(predictions.items(),
                           key = lambda i : i[1], reverse = True)

for i in sorted_predicions[:5]:
    print(f"추천영화 : {movie_names.get(i[0], '제목 없음')} (예측평점 : {i[1]}점)")

추천영화 : Sense and Sensibility (1995) (예측평점 : 5.0점)
추천영화 : Anne Frank Remembered (1995) (예측평점 : 5.0점)
추천영화 : 제목 없음 (예측평점 : 5.0점)
추천영화 : Emma (1996) (예측평점 : 5.0점)
추천영화 : To Gillian on Her 37th Birthday (1996) (예측평점 : 5.0점)


In [22]:
sorted_predicions = sorted(predictions.items(),
                           key = lambda i : i[1], reverse = True)

# 추천할 영화의 수
top_N = 5
recommendations = sorted_predicions[:top_N]

for i in recommendations:
    movie_id = i[0]
    predicted_rating = i[1]

    movie_name = movie_names.get(movie_id, "제목 없음")

    print(f"추천영화 : {movie_name} (예측평점 : {predicted_rating}점)")

추천영화 : Sense and Sensibility (1995) (예측평점 : 5.0점)
추천영화 : Anne Frank Remembered (1995) (예측평점 : 5.0점)
추천영화 : 제목 없음 (예측평점 : 5.0점)
추천영화 : Emma (1996) (예측평점 : 5.0점)
추천영화 : To Gillian on Her 37th Birthday (1996) (예측평점 : 5.0점)
